In [20]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [21]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [22]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model='gpt-3.5-turbo')

In [23]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content='my favourite color is blue')
]

In [24]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and soothing color that represents tranquility and stability. It is often associated with feelings of peace and serenity. It is a versatile color that can be both bold and subtle, making it a popular choice for many people. What do you like most about the color blue?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 12, 'total_tokens': 70, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6b2dc70b-ad13-48e8-ba9d-699521b730de-0', usage_metadata={'input_tokens': 12, 'output_tokens': 58, 'total_tokens': 70})

In [25]:
chatbot.invoke('what is my favourite color?')

AIMessage(content="I'm sorry, I am not able to know your favorite color as I am an AI and do not have access to personal information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 13, 'total_tokens': 40, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6014c4e-86b5-4675-943a-99dd1567b845-0', usage_metadata={'input_tokens': 13, 'output_tokens': 27, 'total_tokens': 40})

## Lets add memory to the chatbot

In [26]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbot_memory =  {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in chatbot_memory:
        chatbot_memory[session_id] = ChatMessageHistory()
    return chatbot_memory[session_id]

chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot,
    get_session_history
)

In [27]:
session1 = { 'configurable': {'session_id': '001'}}

responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='My favourite color is red')],
    config=session1
)
responseFromChatbot.content

"Red is a bold and vibrant color that symbolizes passion, energy, and power. It is often associated with love, strength, and determination. This striking hue can evoke a range of emotions and make a strong visual impact. Whether it's a bold red dress, a vibrant red lipstick, or a fiery red sunset, there's something captivating about this color that always draws me in."

In [28]:
chatbot_memory

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favourite color is red', additional_kwargs={}, response_metadata={}), AIMessage(content="Red is a bold and vibrant color that symbolizes passion, energy, and power. It is often associated with love, strength, and determination. This striking hue can evoke a range of emotions and make a strong visual impact. Whether it's a bold red dress, a vibrant red lipstick, or a fiery red sunset, there's something captivating about this color that always draws me in.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 12, 'total_tokens': 89, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a8075e9-a044-41fb-af57-36c746b76600-0', usage_metadata={'input_tokens': 12, 'output_tokens': 77, 'total_tokens': 89})])}

In [29]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='what is my favorite color?')],
    config=session1
)

In [30]:
responseFromChatbot.content

"I'm not sure what your favorite color is, as I don't have that information. Feel free to share with me your favorite color so I can learn more about your preferences!"

In [31]:
session2 = { 'configurable': {'session_id': '002'}}

responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='what is my favourite color?')],
    config=session2
)
responseFromChatbot.content

"I'm sorry, I do not have access to that information. Can you please tell me your favorite color?"

In [32]:
chatbot_memory['001'].messages

[HumanMessage(content='My favourite color is red', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Red is a bold and vibrant color that symbolizes passion, energy, and power. It is often associated with love, strength, and determination. This striking hue can evoke a range of emotions and make a strong visual impact. Whether it's a bold red dress, a vibrant red lipstick, or a fiery red sunset, there's something captivating about this color that always draws me in.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 12, 'total_tokens': 89, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a8075e9-a044-41fb-af57-36c746b76600-0', usage_metadata={'input_tokens': 12, 'output_tokens': 77, 'total_tokens': 89}),
 HumanMessage(content='what is my favorite color?', additional_kwargs={}, response_

* **The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentialy overflow it**
* **We are now going to learn how to limit the size of the memory of a chatbot**

In [33]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate,  MessagesPlaceholder

def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    print('MESSAGES:',messages[-number_of_messages_to_keep:] )
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [(
        "system",
        "You are a helpful assistant. Answer all questions to the best of your capacities"
    ),
    MessagesPlaceholder(variable_name='messages')]
)

limited_memory_chain = (
    RunnablePassthrough.assign(messages=lambda x : limited_memory_of_messages(x['messages']))
    | prompt
    | chatbot
)

In [34]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limited_memory_chain,
    get_session_history,
    input_messages_key='messages'
)

In [35]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='I like Vespas')],
    config=session1
)
responseFromChatbot.content

"Vespas are stylish and iconic Italian scooters known for their sleek design and vibrant colors. Many Vespas come in a range of eye-catching hues, from classic reds and blues to bold yellows and greens. The color of a Vespa can reflect the owner's personality and style, adding a fun and personalized touch to the scooter. It's great to hear that you enjoy Vespas and appreciate their unique design and colors!"

In [36]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='My favourite ones are red colored')],
    config=session1
)
responseFromChatbot.content

"That's fantastic! Red Vespas are a classic choice and truly make a bold and stylish statement on the road. The vibrant red color adds a touch of flair and sophistication to the Vespa's already chic design. Riding a red Vespa can make you stand out in a crowd and exude confidence and individuality. It's great to hear that you have a preference for red-colored Vespas - it's a timeless choice that never goes out of style!"

In [37]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content='I like cinema')],
    config=session1
)
responseFromChatbot.content

"That's wonderful! Cinema is a fascinating art form that has the power to entertain, inspire, and evoke a wide range of emotions. Whether you enjoy watching blockbuster films, independent movies, foreign films, or documentaries, there is something for everyone in the world of cinema. From classic black and white films to modern-day blockbusters, cinema offers a diverse array of stories, characters, and genres to explore. Do you have a favorite movie or genre that you particularly enjoy watching?"

In [38]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        'messages': [HumanMessage(content='what is my favourite color?')]
    },
    config=session1
)
responseFromChatbot.content

MESSAGES: [AIMessage(content="That's wonderful! Cinema is a fascinating art form that has the power to entertain, inspire, and evoke a wide range of emotions. Whether you enjoy watching blockbuster films, independent movies, foreign films, or documentaries, there is something for everyone in the world of cinema. From classic black and white films to modern-day blockbusters, cinema offers a diverse array of stories, characters, and genres to explore. Do you have a favorite movie or genre that you particularly enjoy watching?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 360, 'total_tokens': 455, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ca1458b-21ff-4853-933f-09ff4ae9507f-0', usage_metadata={'input_tokens': 360, 'output_tokens': 95, 'total_tokens': 455}), HumanMessage(content='what is my fa

"I'm afraid I don't have access to personal information about you, so I wouldn't know what your favorite color is. If you'd like to share your favorite color with me, I'd be happy to discuss it with you or provide information related to that color."

: 